In [ ]:
# import os
# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"

In [ ]:
# isort: off
# stdlib
import os
import sys

# syft absolute
import syft as sy
from syft.util.util import (
    find_base_dir_with_tox_ini,
    get_caller_file_path,
    is_interpreter_jupyter,
)


def add_helper_path_to_python_path() -> None:
    current_path = "."

    # jupyter uses "." which resolves to the notebook
    if not is_interpreter_jupyter():
        # python uses the file which has from syft import test_settings in it
        import_path = get_caller_file_path()
        if import_path:
            current_path = import_path

    base_dir = find_base_dir_with_tox_ini(current_path)
    notebook_helper_path = os.path.join(
        base_dir, "notebooks/scenarios/bigquery/upgradability/0.9.1_helpers"
    )
    sys.path.append(notebook_helper_path)


add_helper_path_to_python_path()

# third party
from email_helpers import Timeout
from email_helpers import get_email_server
# isort: on

In [ ]:
environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")

num_workers = int(os.environ.get("NUM_TEST_WORKERS", 1))

ROOT_EMAIL = "admin@bigquery.org"
ROOT_PASSWORD = "bqpw"
environment

### Launch server & login

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high-migrations",
    dev_mode=True,
    server_side_type="high",
    port="8080",
    n_consumers=num_workers,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [ ]:
email_server, smtp_server = get_email_server(reset=True)

In [ ]:
high_client = sy.login(
    url="http://localhost:8080", email=ROOT_EMAIL, password=ROOT_PASSWORD
)

In [ ]:
high_client.worker_pools

In [ ]:
default_worker_pool = high_client.worker_pools.get_by_name("default-pool")
default_worker_pool

### Scale Worker pool

##### Scale up

In [ ]:
# Scale to 1
if environment == "remote":
    high_client.api.worker_pool.scale(
        number=num_workers, pool_name=default_worker_pool.name
    )

In [ ]:
high_client.api.services.worker_pool[0]

In [ ]:
# Scale up workers
if environment == "remote":
    scale_up_result = high_client.api.worker_pool.scale(
        number=5, pool_name=default_worker_pool.name
    )
    if environment == "remote":
        assert scale_up_result, scale_up_result

        assert (
            high_client.api.services.worker_pool[default_worker_pool.name].max_count
            == 5
        )

##### Scale down

In [ ]:
# Scale down workers, this gracefully shutdowns the consumers
if environment == "remote":
    scale_down_result = high_client.api.worker_pool.scale(
        number=num_workers, pool_name=default_worker_pool.name
    )
    assert scale_down_result, scale_down_result

In [ ]:
if environment == "remote":

    def has_worker_scaled_down():
        return (
            high_client.api.worker_pool[default_worker_pool.name].max_count
            == num_workers
        )

    worker_scale_timeout = Timeout(timeout_duration=20)
    worker_scale_timeout.run_with_timeout(has_worker_scaled_down)

In [ ]:
if environment == "remote":
    assert (
        high_client.api.services.worker_pool[default_worker_pool.name].max_count
        == num_workers
    )

#### Delete Worker Pool

In [ ]:
pool_delete_result = high_client.api.services.worker_pool.delete(
    pool_name=default_worker_pool.name
)
pool_delete_result

In [ ]:
with sy.raises(KeyError):
    _ = high_client.api.services.worker_pool[default_worker_pool.name]

#### Re-launch the default worker pool

In [ ]:
default_worker_image = default_worker_pool.image

In [ ]:
launch_result = high_client.api.services.worker_pool.launch(
    pool_name=default_worker_pool.name,
    image_uid=default_worker_image.id,
    num_workers=num_workers,
)

In [ ]:
assert high_client.api.services.worker_pool[default_worker_pool.name]
assert (
    high_client.api.services.worker_pool[default_worker_pool.name].max_count
    == num_workers
)

In [ ]:
smtp_server.stop()

In [ ]:
server.land()